# Imports

In [ ]:
config = {
    'GRANDE': {
        'split_index_activation': 'entmax',  
        'normalize_index_array': False,
        'split_decision_activation': 'softsign', # sigmoid | softsign
        
        'depth': 6,
        'n_estimators': 2048,
                
        'learning_rate_weights': 0.005,
        'learning_rate_index': 0.01,
        'learning_rate_values': 0.01,
        'learning_rate_leaf': 0.01,   

        'fine_tune': False,
        'cosine_decay_steps': 0,        
        
        'reduce_lr': False,
        'reduce_lr_each_eval_nums': 10,
        'reduce_lr_fraction': 0.2,
        
        'dropout': 0,#0.25,
        
        'selected_variables': 0.8, #int number or float fraction
        'data_subset_fraction': 1.0,
        'bootstrap': False,
                
        'initializer': 'RandomNormal', 
        
        'optimizer': 'SWA', 
        
        'epochs': 10_000,
        'batch_size': 64,#512
        'drop_remainder': True,
        
        
        'loss': 'focal_crossentropy', 
        'focalLossGamma': 2.0,
        
        'transformation_type': 'mean', #for regression; mean log       
        
        'polyLoss': False,
        'polyLossEpsilon': 2.0,
        
        'weight_activation_st': 'softmax',
        'estimator_leaf_weights': True,

        'modify_grads': False,
        
        'logit_weights': False,
        #'weight_freeze_steps': 0,
        'warmup_steps': 0,
        
        'restarts': 0,
        'restart_type': 'loss', #'loss', 'metric'
        
        'early_stopping_epochs': 25,
        'early_stopping_type': 'loss', #'loss', 'metric'
        'early_stopping_epsilon': 1e-3,
        
        'class_weights': True,
        'seed_model': 1,
    },
    
    'preprocessing': {
        'balance_threshold': 0,#.25, #if minclass fraction less than threshold/num_classes | #0=no rebalance, 1=rebalance all
        'normalization_technique': 'quantile', #'min-max'
        'encoding_type': 'LOO', #GLMM
        'one_hot_encode_threshold': 10,
        'quantile_noise': 1e-3,

        'XGBoostEncoding': False,
        'CatBoostEncoding': True,
    },

    'computation': {
        'random_seed': 42,
        'cv_num_eval': 5,
        'subset_size': None, # medium: 10_000, large: 50_000, keep normal size: None
        
        'use_best_hpo_result': True,# True,
        'hpo_path': '_HPO_250_BS-2x5',
        
        'force_depth': False,
        'force_dropout': False,
        'force_class_weights': False,
        'force_restart': True,
        
        'use_gpu': True,
        'gpu_numbers': '0,1,2,3',
        
        'parallel_jobs_cv': 1,
        'jobs_per_gpu': 2,
                
        'search_iterations': 60,
        'cv_num_hpo': 2,     
        'report_test_performance': False,
        
        'metrics_class': ['f1', 'balanced_accuracy', 'roc_auc'],
        'metrics_reg': ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error'],
        
        'eval_metric_class': ['f1', 'balanced_accuracy', 'roc_auc'],
        'eval_metric_reg': ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error'],

        'max_hours': 2.0,   

        'evaluate_GRANDE': True,
    },
    
    'benchmarks': {          
        'CART': False,
        'RandomForest': False,
        
        'XGB': True,
        'CatBoost': True,
        'NODE': True,
    }
}

In [ ]:
import numpy as np
np.set_printoptions(suppress=True)

import sklearn
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, ParameterGrid, ParameterSampler, GridSearchCV, RandomizedSearchCV, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier, plot_tree, DecisionTreeRegressor
from sklearn.metrics import accuracy_score, f1_score, make_scorer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, LabelEncoder, OrdinalEncoder

from livelossplot import PlotLosses

import os
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt

from IPython.display import Image
from IPython.display import display, clear_output

import pandas as pd

if config['computation']['use_gpu']:
    os.environ['CUDA_VISIBLE_DEVICES'] = str(config['computation']['gpu_numbers'])
    os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
    os.environ['XLA_FLAGS'] = "--xla_gpu_cuda_data_dir=/usr/local/cuda-11.6"
    os.environ['TF_XLA_FLAGS'] = "--tf_xla_enable_xla_devices --tf_xla_auto_jit=2"    
    os.environ['PYTORCH_CUDA_ALLOC_CONF'] = "max_split_size_mb:512"    
else:
    os.environ['CUDA_VISIBLE_DEVICES'] = ''
    os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'false' 
#os.environ['TF_XLA_FLAGS'] = "--tf_xla_auto_jit=2 --tf_xla_cpu_global_jit" 


import warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
os.environ["PYTHONWARNINGS"] = "ignore"

import logging
logging.getLogger("ray").setLevel(logging.ERROR)
logging.getLogger("worker").setLevel(logging.ERROR)

import tensorflow as tf
import tensorflow_addons as tfa

tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(3)

np.seterr(all="ignore")

from keras import backend as K

import seaborn as sns
sns.set_style("darkgrid")

import time
import random

from utilities.utilities_GRANDE import *
from utilities.GRANDE import *


from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

from joblib import Parallel, delayed

from itertools import product
from collections.abc import Iterable

from copy import deepcopy
from pathlib import Path
import pickle
import dill

import joblib
import ray
from ray.util.joblib import register_ray
register_ray()
from ray.autoscaler.sdk import request_resources
ray.init()

tf.random.set_seed(config['computation']['random_seed'])
np.random.seed(config['computation']['random_seed'])
random.seed(config['computation']['random_seed'])

from datetime import datetime
timestr = datetime.utcnow().strftime('%Y-%m-%d--%H-%M-%S%f')
print(timestr)
os.makedirs(os.path.dirname("./evaluation_results" + config['computation']['hpo_path'] + "/latex_tables/" + timestr +"/"), exist_ok=True)

filepath = './evaluation_results' + config['computation']['hpo_path'] + '/depth' + str(config['GRANDE']['depth']) + '/' + timestr + '/'
Path(filepath).mkdir(parents=True, exist_ok=True)     


pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num XLA-GPUs Available: ", len(tf.config.experimental.list_physical_devices('XLA_GPU')))
config['computation']['number_of_gpus'] = len(tf.config.experimental.list_physical_devices('GPU'))

In [ ]:
print('Parallel CV Jobs Nested:', config['computation']['parallel_jobs_cv'])
print('Parallel Jobs per GPU:', config['computation']['jobs_per_gpu'])

config['computation']['n_jobs'] = config['computation']['jobs_per_gpu']*config['computation']['parallel_jobs_cv']
config['computation']['num_gpu'] = 1/(np.ceil(config['computation']['n_jobs']/config['computation']['number_of_gpus']))      

config['computation']['verbose'] = 0
n_jobs_outer = max(1, int(np.floor(config['computation']['number_of_gpus'] / 2)))
config['computation']['number_of_gpus'] = config['computation']['number_of_gpus'] / n_jobs_outer

# Hyperparameter Optimization

## Regression

In [ ]:
parameter_dict_GRANDE = {
    
        'depth': tune.choice([4,6]),#tune.randint(5,9),#tune.randint(2,8),
        'n_estimators': tune.choice([512,1024,2048]),#tune.randint(512, 1024),#tune.randint(128, 2048),
    
    
        'learning_rate_weights': tune.loguniform(0.0001, 0.1),
        'learning_rate_index': tune.loguniform(0.005, 0.2),
        'learning_rate_values': tune.loguniform(0.005, 0.2),
        'learning_rate_leaf': tune.loguniform(0.005, 0.2),

        'cosine_decay_steps': tune.choice([0, 100, 1000]),
    
        'loss_reg': tune.choice(['mse']),
        'loss_class': tune.choice(['crossentropy', 'focal_crossentropy']),
    
        'dropout': tune.choice([0.00, 0.25, 0.5]),#tune.uniform(0.0, 0.25),
        'selected_variables': tune.uniform(0.5, 1.0),
        'data_subset_fraction': tune.uniform(0.8, 1.0),
        
        'focalLossGamma': tune.choice([3]),#tune.uniform(1.0, 10.0),
    
        #'polyLoss': tune.choice([True, False]),
        #'polyLossEpsilon': tune.choice([-1, 1, 5, 10, 50]),#tune.uniform(1.0, 100.0),
    
        'optimizer': tune.choice(['SWA']), #'adam', 'SWA', 'GradientAccumulator'
    
        'initializer': tune.choice(['RandomNormal']),
    
        'estimator_leaf_weights': tune.choice([True]),
 
        #'reduce_lr': tune.choice([True, False]),
        #'reduce_lr_each_eval_nums': tune.choice([config['GRANDE']['early_stopping_eval_nums']//2, config['GRANDE']['early_stopping_eval_nums']//3, config['GRANDE']['early_stopping_eval_nums']//4]),
        #'reduce_lr_fraction': tune.choice([0.1, 0.2, 0.5]),
    
        #'warmup_steps': tune.choice([10, 50, 100, 500]),
    
        #'split_index_activation': tune.choice(['softmax', 'entmax', 'sparsemax']), 
        
        #'logit_weights': tune.choice([True, False]),
        'weight_activation_st': tune.choice(['softmax']),
        #'weight_freeze_steps': tune.randint(0, 100),

}

In [ ]:
parameter_dict_CART = {
        'max_depth': tune.randint(2, 12),
        'random_state': tune.choice([config['computation']['random_seed']]),
        'criterion': tune.choice(['gini', 'entropy']), 
        'min_samples_split': tune.randint(2, 50),
        'min_samples_leaf': tune.randint(1, 50),

        'ccp_alpha': tune.loguniform(1e-8, 1.0),
}


In [ ]:
parameter_dict_XGB = {
        'learning_rate': tune.loguniform(0.01, 0.3), #TabSurvey
        'max_depth': tune.randint(2, 12), #TabSurvey
    
        'reg_alpha': tune.loguniform(1e-8, 1.0), #TabSurvey
        'reg_lambda':  tune.loguniform(1e-8, 1.0), #TabSurvey
    
        'n_estimators': tune.choice([1_000]),
        'early_stopping_rounds': tune.choice([20]),    
    
        'max_cat_to_onehot': tune.choice([10]),    
    
        #'learning_rate': tune.loguniform(1e-5, 0.7), #TabularBenchmarks 
        #'max_depth': tune.randint(2, 12), #TabularBenchmarks
        #'gamma': tune.loguniform(1e-8, 7.0), #TabularBenchmarks
        #'max_depth': tune.lograndint(1, 100), #TabularBenchmarks
    
        #'subsample': tune.uniform(0.5, 1.0), #TabularBenchmarks
        #'colsample_bytree': tune.uniform(0.5, 1.0), #TabularBenchmarks 
        #'colsample_bylevel': tune.uniform(0.5, 1.0), #TabularBenchmarks 
    
        #'reg_alpha': tune.loguniform(1e-8, 1e2), #TabularBenchmarks 
        #'reg_lambda':  tune.loguniform(1.0, 4.0), #TabularBenchmarks 

        #'min_child_weight':  tune.quniform(1, 100, 1), #TabularBenchmarks         
    
        'max_cat_threshold': tune.choice([8]),    
    
        'n_jobs': tune.choice([1]),
        'random_state': tune.choice([config['computation']['random_seed']]),
        'verbosity': tune.choice([0]),
    
}   

parameter_dict_CatBoost = {
        'learning_rate': tune.loguniform(0.01, 0.3),#TabSurvey
        'max_depth': tune.randint(2, 12),#TabSurvey
        'l2_leaf_reg': tune.loguniform(0.5, 30.0),#TabSurvey
    
        'n_estimators': tune.choice([1_000]),
        'early_stopping_rounds': tune.choice([20]),

        'random_seed': tune.choice([config['computation']['random_seed']]),
        'verbose': tune.choice([0]),
    
        'gpu_ram_part': tune.choice([0.7/config['computation']['n_jobs']/n_jobs_outer]),
        'used_ram_limit': tune.choice(['16GB']),
        'thread_count': tune.choice([1]),
    
        'boosting_type': tune.choice(['Plain']),
        #'one_hot_max_size': tune.choice([64]),   
        #'leaf_estimation_iterations': tune.choice([1]),   
        #'border_count': tune.choice([256]),  
    
        'devices': tune.choice(['0']),
        #'task_type' 'GPU' / 'CPU'
}   

parameter_dict_RandomForest = {
        'n_estimators': tune.randint(5, 100), #TabSurvey 
    
        'max_depth': tune.randint(2, 12), #TabSurvey 
        'criterion': tune.choice(['gini', 'entropy']), #TabSurvey 

        'n_jobs': tune.choice([1]),
    
        'random_state': tune.choice([config['computation']['random_seed']]),
}   
   



In [ ]:
parameter_dict_NODE = {
        'num_layers': tune.choice([2,4,8]),#TabSurvey
        'total_tree_count': tune.choice([1024, 2048]),#TabSurvey
        'tree_depth': tune.choice([6,8]),#TabSurvey
        #'tree_output_dim': tune.randint(2,3),#TabSurvey
    
        'seed': tune.choice([config['computation']['random_seed']]),
}   


In [ ]:
#CC18 Datasets
#https://github.com/openml/benchmark-suites/blob/44bb11328ef4ae79c67e02ffaf179bd633a5b20b/OpenML%20Benchmark%20generator.ipynb#L1038

identifier_list = [
                        'BIN:CC18_wilt', #40983                                                4,839 |      5 
                        'BIN:CC18_phoneme', #1489                                              5,404 |      5        
                        'BIN:CC18_Amazon_employee_access', #4135                              32,769 |      9              
                        'BIN:CC18_ilpd', #1480                                                   583 |     10 
                        'BIN:CC18_dresses-sales', #23381                                         500 |     12
                        'BIN:CC18_adult', #1590                                               48,842 |     14
                        'BIN:CC18_climate-model-simulation-crashes', #40994                      540 |     18                   
                        'BIN:CC18_churn', #40701                                               5,000 |     20 
                        'BIN:CC18_numerai28.6', #23517                                        96,320 |     21
                        'BIN:CC18_wdbc', #1510                                                   569 |     30 
                        'BIN:CC18_PhishingWebsites', #4534                                    11,055 |     30     
                        'BIN:CC18_cylinder-bands', #6332                                         540 |     37
                        'BIN:CC18_qsar-biodeg', #1494                                          1,055 |     41
                        'BIN:CC18_tokyo1', #40705                                                959 |     44 
                        'BIN:CC18_ozone-level-8hr', #1487                                      2,534 |     72           
                        'BIN:CC18_nomao', #1486                                               34,465 |    118
                        'BIN:CC18_SpeedDating', #40536                                         8,378 |    120
                        'BIN:CC18_madelon', #1485                                              2,600 |    500 
                        'BIN:CC18_Bioresponse', #4134                                          3,751 |  1,776                        
                  ]       

In [ ]:
objective = 'regression' if 'REG:' in identifier_list[0] else 'classification'
parameter_dict_GRANDE['objective'] = tune.choice([objective])

if objective == 'classification':
    config['computation']['eval_metric'] = config['computation']['eval_metric_class'][:1]
    parameter_dict_GRANDE['loss'] = parameter_dict_GRANDE['loss_class']
else:
    config['computation']['eval_metric'] = config['computation']['eval_metric_reg'][:1]
    parameter_dict_GRANDE['loss'] = parameter_dict_GRANDE['loss_reg']
    del parameter_dict_RandomForest['criterion']
    del parameter_dict_CART['criterion']
    
del parameter_dict_GRANDE['loss_reg']
del parameter_dict_GRANDE['loss_class']

#display(parameter_dict)

In [ ]:
def perform_hpo_for_dataset(identifier, 
                            objective,
                            timestr,
                            config, 
                            parallel_jobs_cv,
                            jobs_per_gpu,
                            parameter_dict_GRANDE,
                            parameter_dict_CART,
                            parameter_dict_GeneticTree,
                            parameter_dict_DNDT,
                            parameter_dict_XGB,
                            parameter_dict_CatBoost,
                            parameter_dict_GradientBoostingTree,
                            parameter_dict_LightGBM,
                            parameter_dict_RandomForest,
                            parameter_dict_NODE):
  

    from copy import deepcopy
    import ray
    from ray import tune
    from ray.tune.search.optuna import OptunaSearch    
    import numpy as np
    import pandas as pd
    from ray.tune import JupyterNotebookReporter
    from utilities.utilities_GRANDE import get_preprocessed_dataset, hpo_GRANDE, hpo_CART, hpo_GeneticTree, hpo_DNDT, hpo_XGB, hpo_CatBoost, hpo_GradientBoostingTree, hpo_LightGBM, hpo_RandomForest, hpo_NODE, flatten_list
    import contextlib

    config_hpo = deepcopy(config)
    
    hpo_results_real_world_train = {}
    hpo_results_real_world_unsorted_train = {}
    hpo_results_real_world_valid = {}
    hpo_results_real_world_unsorted_valid = {}
    hpo_results_real_world_test = {}
    hpo_results_real_world_unsorted_test = {}
    hpo_results_real_world_cv = {}
    hpo_results_real_world_unsorted_cv = {}

    hpo_scores_dict_by_dataset = {}
    hpo_scores_dict_by_dataset_max = {}

    scores_df_by_dataset = {}    
    
    #print('______________________________________________________________________________________________________________')
    print('START', identifier, flush=True)

    config_hpo['GRANDE']['objective'] = objective
    
    hpo_scores_dict_by_dataset[identifier] = {'GRANDE': {}}
    benchmark_dict = {}
    
    dataset_dict_list = get_preprocessed_dataset(identifier,
                                                 random_seed=config['computation']['random_seed'],
                                                 config=config_hpo,
                                                 verbosity=0,
                                                 hpo=True)
    
    dataset_dict_list_ref = ray.put(dataset_dict_list)
    
    #print('Parallel CV Jobs Nested:', parallel_jobs_cv)
    #print('Parallel Jobs per GPU:', jobs_per_gpu)
    
    num_gpu = 1/(np.ceil(config['computation']['n_jobs']/config['computation']['number_of_gpus']))      

    if config['computation']['evaluate_GRANDE']:
        hpo_scores_dict_by_dataset[identifier]['GRANDE'] = {}
        config_hpo_benchmark = deepcopy(config_hpo)
        config_hpo_benchmark['computation']['n_jobs'] = config['computation']['n_jobs'] if config_hpo_benchmark['computation']['use_gpu'] else 5
        config_hpo_benchmark['computation']['num_gpu'] = num_gpu*parallel_jobs_cv if config_hpo_benchmark['computation']['use_gpu'] else 0
        benchmark_dict['GRANDE'], hpo_scores_dict_by_dataset[identifier]['GRANDE']['cv'], hpo_scores_dict_by_dataset[identifier]['GRANDE']['test'] = hpo_GRANDE(identifier, dataset_dict_list_ref, parameter_dict_GRANDE, config_hpo_benchmark, metric=config['computation']['eval_metric'][0], greater_better = True, timestr = timestr)
        
    if config['benchmarks']['CART']:
        hpo_scores_dict_by_dataset[identifier]['CART'] = {}
        config_hpo_benchmark = deepcopy(config_hpo)
        config_hpo_benchmark['computation']['n_jobs'] = 3
        benchmark_dict['CART'], hpo_scores_dict_by_dataset[identifier]['CART']['cv'], hpo_scores_dict_by_dataset[identifier]['CART']['test'] = hpo_CART(identifier, dataset_dict_list_ref, parameter_dict_CART, config_hpo_benchmark, metric=config['computation']['eval_metric'][0], greater_better = True, timestr = timestr)
    if config['benchmarks']['XGB']:
        hpo_scores_dict_by_dataset[identifier]['XGB'] = {}
        config_hpo_benchmark = deepcopy(config_hpo)
        config_hpo_benchmark['computation']['n_jobs'] = config['computation']['n_jobs'] if config_hpo_benchmark['computation']['use_gpu'] else 5
        config_hpo_benchmark['computation']['num_gpu'] = num_gpu*parallel_jobs_cv if config_hpo_benchmark['computation']['use_gpu'] else 0
        parameter_dict_XGB['tree_method'] = tune.choice(['gpu_hist']) if config_hpo_benchmark['computation']['use_gpu'] else tune.choice(['hist'])
        benchmark_dict['XGB'], hpo_scores_dict_by_dataset[identifier]['XGB']['cv'], hpo_scores_dict_by_dataset[identifier]['XGB']['test'] = hpo_XGB(identifier, dataset_dict_list_ref, parameter_dict_XGB, config_hpo_benchmark, metric=config['computation']['eval_metric'][0], greater_better = True, timestr = timestr)
    if config['benchmarks']['RandomForest']:
        hpo_scores_dict_by_dataset[identifier]['RandomForest'] = {}
        config_hpo_benchmark = deepcopy(config_hpo)
        config_hpo_benchmark['computation']['n_jobs'] = 3
        benchmark_dict['RandomForest'], hpo_scores_dict_by_dataset[identifier]['RandomForest']['cv'], hpo_scores_dict_by_dataset[identifier]['RandomForest']['test'] = hpo_RandomForest(identifier, dataset_dict_list_ref, parameter_dict_RandomForest, config_hpo_benchmark, metric=config['computation']['eval_metric'][0], greater_better = True, timestr = timestr)
    if config['benchmarks']['NODE']:
        hpo_scores_dict_by_dataset[identifier]['NODE'] = {}
        config_hpo_benchmark = deepcopy(config_hpo)
        config_hpo_benchmark['computation']['n_jobs'] = 1 if config_hpo_benchmark['computation']['use_gpu'] else 3#config['computation']['n_jobs'] if config_hpo_benchmark['computation']['use_gpu'] else 5
        config_hpo_benchmark['computation']['num_gpu'] = num_gpu*parallel_jobs_cv if config_hpo_benchmark['computation']['use_gpu'] else 0
        benchmark_dict['NODE'], hpo_scores_dict_by_dataset[identifier]['NODE']['cv'], hpo_scores_dict_by_dataset[identifier]['NODE']['test'] = hpo_NODE(identifier, dataset_dict_list_ref, parameter_dict_NODE, config_hpo_benchmark, metric=config['computation']['eval_metric'][0], greater_better = True, timestr = timestr)
    if config['benchmarks']['CatBoost']:
        hpo_scores_dict_by_dataset[identifier]['CatBoost'] = {}
        config_hpo_benchmark = deepcopy(config_hpo)
        #config_hpo_benchmark['computation']['use_gpu'] = False
        config_hpo_benchmark['computation']['n_jobs'] = config['computation']['n_jobs'] if config_hpo_benchmark['computation']['use_gpu'] else 5
        config_hpo_benchmark['computation']['num_gpu'] = min(num_gpu*parallel_jobs_cv, 3) if config_hpo_benchmark['computation']['use_gpu'] else 0
        parameter_dict_CatBoost['task_type'] = tune.choice(['GPU']) if config_hpo_benchmark['computation']['use_gpu'] else tune.choice(['CPU'])
        benchmark_dict['CatBoost'], hpo_scores_dict_by_dataset[identifier]['CatBoost']['cv'], hpo_scores_dict_by_dataset[identifier]['CatBoost']['test'] = hpo_CatBoost(identifier, dataset_dict_list_ref, parameter_dict_CatBoost, config_hpo_benchmark, metric=config['computation']['eval_metric'][0], greater_better = True, timestr = timestr)               

    print('DONE', identifier, flush=True)

    return hpo_scores_dict_by_dataset


In [ ]:
with open(os.devnull, "w") as f, contextlib.redirect_stderr(f):
#with contextlib.redirect_stdout(sys.stdout):
    parallel_eval_real_world_nested = Parallel(backend='loky', n_jobs = n_jobs_outer, verbose=13, pre_dispatch=n_jobs_outer) #loky #sequential multiprocessing
    hpo_scores_dict_by_dataset = parallel_eval_real_world_nested(delayed(perform_hpo_for_dataset)(identifier, 
                                                                                    objective,
                                                                                    timestr,
                                                                                    config, 
                                                                                    config['computation']['parallel_jobs_cv'],
                                                                                    config['computation']['jobs_per_gpu'],
                                                                                    parameter_dict_GRANDE,
                                                                                    parameter_dict_CART,
                                                                                    parameter_dict_GeneticTree,
                                                                                    parameter_dict_DNDT,
                                                                                    parameter_dict_XGB,
                                                                                    parameter_dict_CatBoost,
                                                                                    parameter_dict_GradientBoostingTree,
                                                                                    parameter_dict_LightGBM,                                                                               
                                                                                    parameter_dict_RandomForest,
                                                                                    parameter_dict_NODE) for identifier in tqdm(identifier_list))

In [ ]:
from numba import cuda 
if config['computation']['use_gpu']:
    device = cuda.get_current_device()
    device.reset()